I found several restrictions in using the web API by Reddit described in the GA video.

    1. There is a limit of 1000 posts max you can scrap.
    2. Each time only 

In [10]:
import numpy as np
import pandas as pd

import random
import time

from datetime import datetime

import requests

from utils.NLPPost import NLPPost

random.seed(19)

user_agents = []
user_agents_max_idx = 0;

url_subreddit = 'https://www.reddit.com/r/'

url_subreddit_nba = url_subreddit + 'nba.json'
url_subreddit_worldnews = url_subreddit + 'worldnews.json'
url_subreddit_boardgames = url_subreddit + 'boardgames.json'
url_subreddit_gaming = url_subreddit + 'gaming.json'
url_subreddit_starwars = url_subreddit + 'starwars/new.json'
url_subreddit_startrek = url_subreddit + 'startrek.json'

In [ ]:
def load_user_agents():
    
    global user_agents
    global user_agents_max_idx
    
    df = pd.read_csv('user-agents.csv', index_col='id')

    user_agents = list(df['user_agent'])
    
    user_agents_max_idx = len(user_agents) - 1
    
    
def get_random_user_agent():
    
    global user_agents
    global user_agents_max_idx
    
    return user_agents[random.randint(0, user_agents_max_idx)]

In [ ]:
def get_headers():
    
    user_agent = get_random_user_agent()
    
    return {'User-agent': user_agent}

def get_parameters(param_before=None, param_after=None):
     
    parameters = {}
    
    if param_after is not None:
        parameters['after'] = param_after
        
    if param_before is not None:
        parameters['before'] = param_before
        
    print(parameters)
    
    return parameters

def process_response_json(res_json):
    
    posts = []    
    
    posts_before = res_json['data']['before'] 
    posts_after = res_json['data']['after'] 
    
    children = res_json['data']['children']
    
    for child in children:
        
        child_data = child['data']
        
        print(type(child_data))
        
        post = RedditPost(post_dict=child_data)
        
        '''
        name = child_data['name']
        title = child_data['title']
        text = child_data['selftext']
        created = child_data['created_utc']

        posts.append({'name': name, 'title': title, 'text': text, 'created_utc': created, 'before': posts_before, 'after': posts_after})
        '''
        
        posts.append(post)
        
        
    return posts_before, posts_after, posts

In [ ]:
def process_successful_response(response, all_posts, scrap_count, before, after):
    
    next_before, next_after, posts = process_response_json(response.json())
        
    for post in posts:
        all_posts[post.name] = post

    posts_count = len(posts)

    scrap_count = scrap_count + posts_count
    
    if before is not None:
        print(f'Got {posts_count} posts before {before}. Scrapped: {scrap_count}, Unique: {len(all_posts)}')
    else:
        print(f'Got {posts_count} posts after {after}. Scrapped: {scrap_count}, Unique: {len(all_posts)}')
          
    time.sleep(3)
          
    return all_posts, scrap_count, next_before, next_after
    

def get_posts(url, num_posts=0, num_requests=0, before=None, after=None):
    
    all_posts = {}
    posts_before = before
    posts_after = after
    
    request_count = 0
    posts_count = 0
    scrap_count = 0
    
    while len(all_posts) < num_posts:
        
        response = requests.get(url, headers=get_headers(), params=get_parameters(param_before=posts_before, param_after=posts_after))

        code = response.status_code
        
        request_count+=1
        
        print(f'Request #{request_count}: code={code}')
        
        if code == 200:
             all_posts, scrap_count, posts_before, posts_after = process_successful_response(response, all_posts, scrap_count, posts_before, posts_after)
          
    
    for i in range(num_requests):

        response = requests.get(url, headers=get_headers(), params=get_parameters(param_before=posts_before, param_after=posts_after))

        code = response.status_code

        print(f'Request #{i+1}: code={code}')

        if code == 200:
            all_posts, scrap_count, posts_before, posts_after = process_successful_response(response, all_posts, scrap_count, posts_before, posts_after)
    
    return posts_before, posts_after, all_posts

In [ ]:
load_user_agents()

posts = {}

next_before, next_after, posts = get_posts(url_subreddit_starwars, num_requests=3)

In [827]:
print(posts_after)

t3_e4boay


In [828]:
df = pd.DataFrame(list(posts.values()))

len(df['after'].unique())

39

In [829]:
df.to_csv('starwars1.csv', index=False)

In [830]:
df = pd.read_csv('starwars1.csv')

df['created'] = [ datetime.utcfromtimestamp(created) for created in df['created_utc']]
df.sort_values(by='created', ascending=True)

,after,before,created_utc,name,text,title,created
974,t3_e3a6eu,NaN,1.575005e+09,t3_e3a6eu,"I get it, people are mad at Episode VIII. But ...",I'm sick of seeing people nitpicking Episode I...,2019-11-29 05:15:51
973,t3_e3a6eu,NaN,1.575005e+09,t3_e3a85a,I know I've made multiple posts about this sub...,Question about correct viewing order,2019-11-29 05:20:14
972,t3_e3a6eu,NaN,1.575005e+09,t3_e3a8gp,Disney + has let me watch them now!,Should I watch the Clone Wars or Rebels first?,2019-11-29 05:20:48
971,t3_e3a6eu,NaN,1.575005e+09,t3_e3aaim,NaN,[OC] Star Wars Episode VI – Return of the Jedi...,2019-11-29 05:26:06
970,t3_e3a6eu,NaN,1.575005e+09,t3_e3abt7,NaN,ROTJ's ending in hindsight,2019-11-29 05:29:27
969,t3_e3a6eu,NaN,1.575006e+09,t3_e3actx,NaN,I tried my best :|,2019-11-29 05:32:14
968,t3_e3a6eu,NaN,1.575006e+09,t3_e3afbs,"Anyone have any collections, canon or EU, that...",Book Collections,2019-11-29 05:38:51
967,t3_e3a6eu,NaN,1.575006e+09,t3_e3ah6w,NaN,Are the prequels really that bad or are people...,2019-11-29 05:43:56
966,t3_e3a6eu,NaN,1.575006e+09,t3_e3aikv,NaN,Epic shot: one of my favorite BTS photos in th...,2019-11-29 05:47:44
965,t3_e3a6eu,NaN,1.575007e+09,t3_e3ak79,NaN,Red five standing bye,2019-11-29 05:52:09


In [831]:
df.to_csv('startrek2_utc.csv', index=False)

In [819]:
df['created'].min(), df['created'].max()

(Timestamp('2019-11-29 04:24:55'), Timestamp('2019-12-02 14:15:31'))

In [832]:
df['created'].min(), df['created'].max()

(Timestamp('2019-11-29 05:15:51'), Timestamp('2019-12-02 14:36:22'))

In [733]:
df1 = pd.read_csv('startrek1_utc.csv')
df2 = pd.read_csv('startrek2_utc.csv')

df1.shape[0] + df2.shape[0]


1929

In [734]:
combined = list(df1['name'])

combined.extend(list(df2['name']))


print(len(set(combined)))

977


In [847]:
def merge_unique_posts(list_posts):
    
    dict_merged = {}
    
    for i in range(len(list_posts)):
        
        dict_posts = list_posts[i].T.to_dict()
        
        dict_merged.update(dict_posts)
            
    return dict_merged


In [848]:
df1 = pd.read_csv('startrek1_utc.csv')
df2 = pd.read_csv('startrek2_utc.csv')

names1 = list(df1['name'])
names2 = list(df2['name'])

print(len(names1), len(names2))

names1.extend(names2)
names = set(names1)

len(names)

992 975


997

In [849]:


dict_all_posts = merge_unique_posts([df1, df2])

print(len(dict_all_posts))

#print(dict_all_posts)

df_merged = pd.DataFrame(list(dict_all_posts.values()))

df_merged.head()


992


,after,before,created,created_utc,name,text,title
0,t3_e4ynkv,NaN,2019-12-02 14:36:22,1.575297e+09,t3_e4zwfp,NaN,Help me Obi Wan.
1,t3_e4ynkv,NaN,2019-12-02 14:35:03,1.575297e+09,t3_e4zvun,NaN,I realized that the sequels aren't numbered on...
2,t3_e4ynkv,NaN,2019-12-02 14:26:18,1.575297e+09,t3_e4zrzu,Re-watched TLJ for the first time yesterday. ...,The Last Jedi: Luke's Lightsaber
3,t3_e4ynkv,NaN,2019-12-02 14:24:59,1.575297e+09,t3_e4zrdf,"No access to Disney+, baby Yoda literally ever...",Am I the only frustrated non-American?
4,t3_e4ynkv,NaN,2019-12-02 14:24:15,1.575297e+09,t3_e4zr1h,But where are all my Battle of the Heroes peop...,I know Duel of the Fates is amazing and all...


In [846]:
df_merged.shape

(992, 7)

In [65]:
def post_to_words(post):
    
    contents = post.title + ' ' + post.text

    # Remove HTML tags
    contents = BeautifulSoup(contents).get_text()
    
    # Remove digits and special characters
    letters_only = re.sub("[^a-zA-Z]", " ", contents)
    
    # Convert to lower case and split into individual words
    words = letters_only.lower().split()
    
    # Get stop words
    stops = set(stopwords.words('english'))
    
    meaningful_words = [word for word in words if not word in stops]
    
    return " ".join(meaningful_words)

In [843]:
datetime.datetime.timestamp(datetime.datetime.utcnow())
#datetime.datetime.utcfromtimestamp(1514764800)

1575331408.118855

In [62]:
def remove_emojis(text):
    
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "]+", flags=re.UNICODE)
    
    return emoji_pattern.sub(r'', text)

In [11]:
import praw
import pprint

reddit = praw.Reddit(client_id='decGUQHbZSwTxA', client_secret='ePMxrPWEiouDDV07dZiVpJ08y4g', user_agent='project3')

sr = reddit.subreddit('starwars')

posts = sr.new(limit=5)

for post in posts:
    
    nlp_post = NLPPost(post.name, post.subreddit, post.title, post.selftext, post.created_utc)
    
    nlp_post.save_to_dynamodb()
        
    #pprint.pprint(vars(post))

ResourceNotFoundException: An error occurred (ResourceNotFoundException) when calling the DescribeTable operation: Requested resource not found: Table: posts not found

In [14]:
'dvvdsz'.split(' ')

['dvvdsz']